## Experiments

Let's start playing around with our data

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tqdm
from sklearn.model_selection import train_test_split
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification,AutoModel,AutoConfig
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from imblearn.under_sampling import RandomUnderSampler
from torch.optim import Adam, AdamW
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-04-27 00:25:01.561570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 00:25:03.151595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Read in training data
This dataset contains 100 annotated terms of service contracts, each row represents a sentence, which carries on it a label. The label corresponds to a different type of potential unfairness, as defined by the authors of CLAUDETTE, the previous paper from which this dataset came from. 

In [2]:
df = pd.read_csv('../data/dataset.csv')
df.head()

,Unnamed: 0,A,CH,CR,J,LAW,LTD,PINC,TER,USE,document,document_ID,label,text,TER_targets,LTD_targets,A_targets,CH_targets,CR_targets
0,0,0,0,0,0,0,0,0,0,0,Mozilla,0,0,websites & communications terms of use,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,Mozilla,0,0,please read the terms of this entire document ...,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,1,Mozilla,0,1,by accessing or signing up to receive communic...,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,Mozilla,0,0,our websites include multiple domains such as ...,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,Mozilla,0,0,you may also recognize our websites by nicknam...,NaN,NaN,NaN,NaN,NaN


Before we were trying binary classification, which wasn't producing great results, let's see if we can get better results using the individual types of labels as classified by the text. The logic here is that because each type of potential unfairness likely has some semantic differences, conglomerating them all into one made it difficult to pick them all out. 

# Multi-Label Preprocessing

In [3]:
df['labels'] = df.apply(lambda row: (1 if row['A'] == 1 else 2 if row['CH'] == 1 else 3 if row['CR'] == 1 else 4 if row['J'] == 1 else 5 if row['LAW'] == 1 else 6 if row['LTD'] == 1 else 7 if row['PINC'] == 1 else 8 if row['TER'] == 1 else 9 if row['USE'] == 1 else 0),axis=1)
x_multi = df['text']
y_multi = df['labels']
df['labels'].value_counts(normalize=True)

labels
0    0.893324
6    0.029681
2    0.016849
8    0.015085
9    0.011853
3    0.010286
4    0.006661
5    0.006122
1    0.005192
7    0.004947
Name: proportion, dtype: float64

In [4]:
label2id = {
    'FAIR':0,
    'A':1,
    'CH':2,
    'CR':3,
    'J':4,
    'LAW':5,
    'LTD':6,
    'PINC':7,
    'TER':8,
    'USE':9
}
id2label = {v:k for k,v in label2id.items()}
id2label

{0: 'FAIR',
 1: 'A',
 2: 'CH',
 3: 'CR',
 4: 'J',
 5: 'LAW',
 6: 'LTD',
 7: 'PINC',
 8: 'TER',
 9: 'USE'}

# Binary Label Pre-Processing

In [5]:
x, y = df['text'], df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)
rus = RandomUnderSampler(sampling_strategy=1)
x_train_res, y_train_res = rus.fit_resample(pd.DataFrame(x_train), pd.DataFrame(y_train))

In [6]:
y_train_res.value_counts()

label
0        1730
1        1730
Name: count, dtype: int64

In [7]:
train_df = pd.merge(x_train_res,y_train_res,left_index=True,right_index=True)
test_df = pd.merge(x_test,y_test,left_index=True,right_index=True)
val_df = pd.merge(x_val,y_val,left_index=True,right_index=True)
train_df = train_df.sample(frac=1)
subset_df = train_df.sample(50)
test_subset_df = test_df.sample(50)

In [8]:
dataset_dict = DatasetDict(
    {
    "train":Dataset.from_pandas(train_df),
    "test":Dataset.from_pandas(test_df),
    "val":Dataset.from_pandas(val_df),
    "train_subset":Dataset.from_pandas(subset_df),
    "test_subset":Dataset.from_pandas(test_subset_df)
    }
)

Split dataset into train, test and validation

In [9]:
tokenizer = BertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
def tokenize(batch):
    return tokenizer(batch['text'], max_length=512,padding='max_length')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [10]:
tokenized_dict = dataset_dict.map(tokenize, batched=True, batch_size=len(dataset_dict))
tokenized_dict.set_format('torch', columns=['input_ids', 'attention_mask',"token_type_ids",'label'])

Map:   0%|          | 0/3460 [00:00<?, ? examples/s]

Map:   0%|          | 0/2042 [00:00<?, ? examples/s]

Map:   0%|          | 0/2042 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Now let's construct a custom classifier classifier to classify sentences as potentially unfair or not 

In [11]:
class Classifier_v1(nn.Module):
    def __init__(self, model_name):
        super(Classifier_v1,self).__init__()
        self.model = AutoModel.from_pretrained(model_name,config = AutoConfig.from_pretrained(model_name,
                                                                                              output_attention = True,
                                                                                              output_hidden_state = True))
        # New Layer
        self.dropout = nn.Dropout(0.1)
        self.hidden = nn.Linear(self.model.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, input = None, attention_mask = None, token_type_ids = None,input_ids = None, labels = None):
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        pooler_output = outputs.pooler_output[0]
        output = self.hidden(pooler_output)
        logit = self.sigmoid(output)
        return logit

In [12]:
from torch.utils.data import DataLoader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(
    tokenized_dict['train'], shuffle=True, batch_size=1, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_dict['test'], shuffle=True, batch_size=1,collate_fn=data_collator
)
val_dataloader = DataLoader(
    tokenized_dict['val'], shuffle=True, batch_size=1,collate_fn=data_collator
)
train_subset_dataloader = DataLoader(
    tokenized_dict['train_subset'], shuffle=True, batch_size=1,collate_fn=data_collator
)
test_subset_dataloader = DataLoader(
    tokenized_dict['test_subset'], shuffle=True, batch_size=1,collate_fn=data_collator
)

## Training Loop

In [13]:
torch.cuda.empty_cache()

# Full Loop

In [14]:
def train(model,train_dataloader,test_dataloader,epochs,loss_fn,optimizer,num_train_samples,num_test_samples,lr_scheduler=None,testing=False):
    for epoch in range(epochs):
        model.train()
        train_labels = []
        train_preds = []
        train_loss = 0
        for batch in tqdm.tqdm(train_dataloader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            label = batch['labels'].to(torch.float32).to(device)
            output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            loss = loss_fn(output[0],label)
            train_loss += loss.item()
            pred = torch.round(output[0])
            train_preds.append(pred.detach().cpu().numpy())
            train_labels.append(label.detach().cpu().numpy())
            loss.backward()
            optimizer.step()
            if lr_scheduler:
                lr_scheduler.step(loss)
        print(f"Train Epoch: {epoch + 1} | Accuracy: {accuracy_score(train_labels,train_preds)} | Precision: {precision_score(train_labels,train_preds)} | Recall: {recall_score(train_labels,train_preds)} | F1: {f1_score(train_labels,train_preds)} | Loss: {train_loss/num_train_samples}")
        if testing:
            preds = []
            labels = []
            test_loss = 0
            for batch in tqdm.tqdm(test_dataloader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                label = batch['labels'].to(torch.float32).to(device)
                with torch.no_grad():
                    output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
                loss = loss_fn(output[0],label)
                test_loss += (loss.item())
                pred = torch.round(output[0])
                preds.append(pred.detach().cpu().numpy())
                labels.append(batch["labels"].detach().cpu().numpy())
            print(f"Test Epoch: {epoch + 1} | Accuracy: {accuracy_score(labels,preds)} | Precision: {precision_score(labels,preds)} | Recall: {recall_score(labels,preds)} | F1: {f1_score(labels,preds)} | Loss: {test_loss/num_test_samples}")

In [15]:
def test(model,test_dataloader,loss_fn,num_samples):
    model.eval()
    raw_preds = []
    preds = []
    labels = []
    test_loss = 0
    for batch in tqdm.tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label = batch['labels'].to(torch.float32).to(device)
        with torch.no_grad():
            output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        loss = loss_fn(output[0],label)
        test_loss += (loss.item())
        raw_preds.append(output[0].detach().cpu().numpy())
        pred = torch.round(output[0])
        preds.append(pred.detach().cpu().numpy())
        labels.append(batch["labels"].detach().cpu().numpy())
    print(f"Accuracy: {accuracy_score(labels,preds)} | Precision: {precision_score(labels,preds)} | Recall: {recall_score(labels,preds)} | F1: {f1_score(labels,preds)} | Loss: {test_loss/num_samples}")
    return raw_preds,preds,labels

Looking much better than before, but there is still a lot of room for improvement

This is not working, even with oversampling we are still not getting good results. I think the problem is that the pooler output, which is the (mean?) of all the 12 hidden states of BERT that we are using as input to our additional classifier is not capturing the information we need to understand the fairness of a sentence. Instead of using the pooler output, let's use a concatenation of all the hidden states of the BERT model

In [16]:
class Classifier_V2(nn.Module):
    '''
    This model is similar to the first one but instead of using the pooler output, it uses the hidden states of the model
    The 'hidden_states_used' parameter is used to determine how many hidden states to use, smaller values of this will be less computationally expensive, but likely less accurate
    '''
    def __init__(self, model_name ,num_labels,hidden_states_used):
        super(Classifier_V2,self).__init__()
        self.hidden_states_used = hidden_states_used
        self.model = BertModel.from_pretrained(model_name,config = BertConfig.from_pretrained(model_name,output_hidden_states = True,num_labels=num_labels))
        self.hidden1 = nn.Linear(self.model.config.hidden_size*self.model.config.max_position_embeddings*self.hidden_states_used, 64)
        self.hidden_p = nn.Linear(self.model.config.hidden_size, 64)
        self.fc = nn.Linear(64, num_labels)
        self.dropout = nn.Dropout(0.1)
        if num_labels == 1:
            self.activation = nn.Sigmoid()
        else:
            self.activation = nn.Softmax(dim=1)
    """
    In the forward function we take the hidden states of the model and concatenate them along the first dimension, and then we pass them through a linear layer
    We also take the pooler output of the sentence and pass it through a separate linear layer
    We then add the two outputs together and pass them through a final linear layer to classify the output
    We do this in hopes that the model will learn to use the hidden states to learn more about the semantic meaning of the text,
    while the pooler output will learn more about the overall meaning of the text
    """
    def forward(self, attention_mask = None, token_type_ids = None,input_ids = None):
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        hidden_states = torch.cat(outputs.hidden_states[-self.hidden_states_used:],dim=0).view(1,-1)
        pooler_output = outputs.pooler_output
        x_pooler = self.hidden_p(self.dropout(pooler_output))
        x_hidden = self.hidden1(self.dropout(hidden_states))
        x = torch.add(x_pooler,x_hidden)
        output = self.fc(x)
        logit = self.activation(output)
        return logit

In [17]:
model_v5 = Classifier_V2('distilbert/distilbert-base-uncased',1,6).to(device)
loss_fn = nn.BCELoss()
optimizer = Adam(model_v5.parameters(),lr =.00000005)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.l

In [18]:
model_v5.load_state_dict(torch.load('../models/distil_bert_6_.pth'))

<All keys matched successfully>

In [19]:
train(model_v5,train_dataloader,test_dataloader,6,loss_fn,optimizer,num_train_samples=3460,num_test_samples=2042,testing=True)

100%|██████████| 3460/3460 [51:00<00:00,  1.13it/s] 


Train Epoch: 1 | Accuracy: 0.7569364161849711 | Precision: 0.7515563101301641 | Recall: 0.7676300578034682 | F1: 0.7595081498427223 | Loss: 0.49785690285878476


100%|██████████| 2042/2042 [09:12<00:00,  3.69it/s]


Test Epoch: 1 | Accuracy: 0.6077375122428991 | Precision: 0.20591233435270132 | Recall: 0.9017857142857143 | F1: 0.33526970954356844 | Loss: 0.6737446827778865


100%|██████████| 3460/3460 [44:02<00:00,  1.31it/s] 


...

Test Epoch: 5 | Accuracy: 0.7830558276199804 | Precision: 0.30127041742286753 | Recall: 0.7410714285714286 | F1: 0.42838709677419357 | Loss: 0.4726684592494579


100%|██████████| 3460/3460 [42:57<00:00,  1.34it/s] 


Train Epoch: 6 | Accuracy: 0.7809248554913295 | Precision: 0.7786697247706422 | Recall: 0.784971098265896 | F1: 0.7818077144502015 | Loss: 0.46818796901477805


100%|██████████| 2042/2042 [09:47<00:00,  3.47it/s]

Test Epoch: 6 | Accuracy: 0.9020568070519099 | Precision: 0.6132075471698113 | Recall: 0.29017857142857145 | F1: 0.3939393939393939 | Loss: 0.279273193060911


In [22]:
results = test(model_v5,val_dataloader,loss_fn,2042)

100%|██████████| 2042/2042 [13:44<00:00,  2.48it/s]

Accuracy: 0.910871694417238 | Precision: 0.7674418604651163 | Recall: 0.2894736842105263 | F1: 0.42038216560509556 | Loss: 0.2507265433188207


In [21]:
torch.save(model_v5.state_dict(),'../models/distil_bert_6_.pth')

In [22]:
# del loss_fn
# del optimizer
# del model_v2
#del scheduler
torch.cuda.empty_cache()